## Домашнее задание

### 1. Загрузите тренировочные и тестовые датасеты

In [1]:
import pandas as pd
#from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split

/home/w_lander/anaconda3/envs/dowgrade/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Ваш код здесь
train = pd.read_csv('data/TrainData.csv')
test = pd.read_csv('data/TestData.csv')

In [3]:
train.shape, test.shape

((7500, 15), (2500, 15))

### 2. Оцените баланс классов в задаче
- Затем попытайтесь устно ответить на вопрос, можно ли использовать accuracy как метрику качества в задаче?

In [4]:
# Ваш код здесь
train['target'].value_counts()

target
0    5708
1    1792
Name: count, dtype: int64

In [5]:
# для дисбаланса классов, метрика accuracy не подходит. Необходимо использовать precision или recall в зависимости от задачи

### 3. Постройте baseline-модель:
- разбейте TrainData на тренировочные (Train) и тестовые данные (Test);
- обучите LogisticRegression и SVC с параметрами по умолчанию на тренировочных данных (Train);
- примените модели на тестовых данных (Test).

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   f1      7500 non-null   int64  
 1   f2      7425 non-null   float64
 2   f3      7500 non-null   int64  
 3   f4      7500 non-null   float64
 4   f5      7500 non-null   int64  
 5   f6      7500 non-null   float64
 6   f7      5625 non-null   float64
 7   f8      7500 non-null   float64
 8   f9      7500 non-null   float64
 9   f10     7500 non-null   float64
 10  f11     7490 non-null   float64
 11  f12     7500 non-null   int64  
 12  f13     7500 non-null   int64  
 13  f14     7500 non-null   float64
 14  target  7500 non-null   int64  
dtypes: float64(9), int64(6)
memory usage: 879.0 KB


In [7]:
# Ваш код здесь
X = train.drop(['target', 'f2', 'f7', 'f11'], axis=1)
Y = train['target']
X_scaled = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, train_size=0.3, random_state=42)

In [8]:
X_train.shape, y_train.shape

((2250, 11), (2250,))

In [9]:
model_log_r = LogisticRegression().fit(X_train, y_train)

pred_test = model_log_r.predict(X_test)

print('TEST LogisticRegression')
print(classification_report(y_test, pred_test))

TEST LogisticRegression
              precision    recall  f1-score   support

           0       0.85      0.91      0.88      4000
           1       0.63      0.49      0.55      1250

    accuracy                           0.81      5250
   macro avg       0.74      0.70      0.72      5250
weighted avg       0.80      0.81      0.80      5250



In [10]:
model_svc = SVC().fit(X_train, y_train)

pred_train = model_svc.predict(X_train)

pred_test = model_svc.predict(X_test)

print('TEST model_svc')
print(classification_report(y_test, pred_test))

TEST model_svc
              precision    recall  f1-score   support

           0       0.84      0.94      0.89      4000
           1       0.69      0.42      0.52      1250

    accuracy                           0.82      5250
   macro avg       0.76      0.68      0.70      5250
weighted avg       0.80      0.82      0.80      5250



In [11]:
# Обе модели справились примерно одинаково, и не очень хорошо определяюи 1 класс, без балансировки классов

### 4. Улучшите модели
Попробуйте улучшить качество обученных моделей:
- можете задавать class_weights;
- можете изменять параметры модели;
- можете вручную или при помощи методов Python генерировать новые признаки и/или удалять существующие.

Это самая важная и творческая часть задания. Проводите как можно больше экспериментов!

Проведите минимиум три эксперимента: для каждого типа модели минимум один эксперимент.

In [12]:
model_log_r = LogisticRegression(class_weight='balanced').fit(X_train, y_train)

pred_test = model_log_r.predict(X_test)

print('TEST balanced LogisticRegression')
print(classification_report(y_test, pred_test))

TEST balanced LogisticRegression
              precision    recall  f1-score   support

           0       0.94      0.75      0.83      4000
           1       0.51      0.84      0.64      1250

    accuracy                           0.77      5250
   macro avg       0.72      0.79      0.74      5250
weighted avg       0.84      0.77      0.79      5250



In [13]:
model_svc = SVC(class_weight='balanced').fit(X_train, y_train)

pred_test = model_svc.predict(X_test)

print('TEST balanced model_svc')
print(classification_report(y_test, pred_test))

TEST balanced model_svc
              precision    recall  f1-score   support

           0       0.94      0.74      0.83      4000
           1       0.51      0.86      0.64      1250

    accuracy                           0.77      5250
   macro avg       0.73      0.80      0.73      5250
weighted avg       0.84      0.77      0.78      5250



### 5. Оцените на отложенной выборке качество наилучшей модели
В пунктах 3 и 4 вы построили много разных моделей.

Возьмите ту, которая дала наилучшее качество на тестовых данных (Test). Примените её на отложенной выборке (TestData) и выведите на экран значение метрики f1.

In [14]:
def_test = test.drop(['target', 'f2', 'f7', 'f11'], axis=1)
def_y = test['target']
def_test_scaled = StandardScaler().fit_transform(def_test)

In [15]:
pred_test = model_svc.predict(def_test_scaled)

print('TEST balanced model_svc')
print(classification_report(def_y, pred_test))

TEST balanced model_svc
              precision    recall  f1-score   support

           0       0.89      0.66      0.76      1913
           1       0.39      0.72      0.51       587

    accuracy                           0.67      2500
   macro avg       0.64      0.69      0.63      2500
weighted avg       0.77      0.67      0.70      2500



### 6. Выполните хитрый трюк
Часто смешивание различных моделей даёт улучшение итогового предсказания. Попробуйте смешать две лучшие модели по формуле:
$$pred_{final} = \alpha\cdot pred_1 + (1-\alpha)\cdot pred_2$$.

Значение $\alpha$ подберите в цикле по Test-выборке. Оцените качество на отложенной выборке.

Удалось ли добиться улучшения качества?

In [16]:
# Ваш код здесь

### 7. Сделайте выводы

Запишите в отдельной ячейке текстом выводы о проделанной работе. Для этого ответьте на вопросы:
- Какие подходы вы использовали для улучшения работы baseline-моделей?
- Какого максимального качества удалось добиться на Test-данных?
- Какое при этом получилось качество на отложенной выборке?
- Ваша модель переобучилась, недообучилась или обучилась как надо?

In [17]:
# Модель немлохо определяет 0 класс и  1 класс, балансировка улучшила результаты, но на достаточно большой выборке модель училась на одинаковых наблюдениях.
# Можно попробовать более сложные модели, а также создать новые признаки

Важный комментарий! В реальных задачах не следует ожидать, что машинным обучением всегда удастся решить задачу с хорошим качеством. Но использовать все имеющиеся у вас в арсенале методы для достижения наилучшего результата нужно.